In [1]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import xgboost, lightgbm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split, KFold, cross_val_score

import os
import time
import gc
import warnings

warnings.filterwarnings('ignore')


In [2]:
train_X = pd.read_csv('./train_X.csv')
test_X = pd.read_csv('./test_X.csv')

train_y = pd.read_csv('./train_y.csv')
m, s = np.mean(train_y), np.std(train_y)

sub = pd.read_csv('./data/sample_submission.csv')

In [3]:
train_y = (train_y - np.mean(train_y))/np.std(train_y)

In [7]:
xgb_bounds = {
    'max_depth': (3, 12),
    'subsample': (0.3, 1),
    'colsample_bytree': (0.3, 1),
    'reg_alpha': (0.1, 3),
    'reg_lamda': (0.1, 3)
}

best_ns = []

def bayes_xgb(max_depth, subsample, colsample_bytree, reg_alpha, reg_lamda):
    global best_ns
    params = {
        'n_estimators': 1000,
        'booster': 'gbtree',
        'tree_method': 'gpu_hist',
        'random_state': 0,
        'n_jobs': -1,
        'max_depth': int(max_depth),
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'reg_alpha': reg_alpha,
        'reg_lamda': reg_lamda
    }
    
    acc = 0
    best_n = 0
    for tr_idx, val_idx in kfold.split(train_X):
        tr_X, tr_y = train_X.iloc[tr_idx, :], train_y.iloc[tr_idx]
        val_X, val_y = train_X.iloc[val_idx, :], train_y.iloc[val_idx]
        clf = XGBRegressor(**params, objective='reg:squarederror').fit(tr_X, tr_y, eval_metric = 'error', eval_set=[[val_X, val_y]], early_stopping_rounds=100, verbose=0)
        acc -= mean_squared_error(val_y, clf.predict(val_X))/10
        best_n += clf.best_iteration/10
    best_ns.append(best_n)
    return acc


In [8]:
kfold = KFold(n_splits=10, shuffle=True, random_state=0)

In [9]:
%%time
optimizer = BayesianOptimization(bayes_xgb, xgb_bounds, random_state=0)

init_points = 10
n_iter = 20

optimizer.maximize(init_points=init_points, n_iter=n_iter)

|   iter    |  target   | colsam... | max_depth | reg_alpha | reg_lamda | subsample |
-------------------------------------------------------------------------------------
|  1        | -0.291    |  0.6842   |  9.437    |  1.848    |  1.68     |  0.5966   |
|  2        | -0.2461   |  0.7521   |  6.938    |  2.686    |  2.895    |  0.5684   |
|  3        | -0.2217   |  0.8542   |  7.76     |  1.747    |  2.784    |  0.3497   |
|  4        | -0.1603   |  0.361    |  3.182    |  2.515    |  2.357    |  0.909    |
|  5        | -0.2427   |  0.985    |  10.19    |  1.438    |  2.364    |  0.3828   |
|  6        | -0.1348   |  0.7479   |  4.29     |  2.84     |  1.613    |  0.5903   |
|  7        | -0.1554   |  0.4852   |  9.968    |  1.423    |  1.748    |  0.3132   |
|  8        | -0.2581   |  0.7323   |  8.509    |  1.889    |  2.837    |  0.7773   |
|  9        | -0.2207   |  0.5517   |  6.933    |  2.123    |  0.2747   |  0.7667   |
|  10       | -0.121    |  0.7694   |  4.893    |  0.4

In [ ]:
import pickle
with open('./best_params.bin', 'rb') as f:
    best_params = pickle.load(f)

In [ ]:
optimizer.max['params']

In [10]:
best_params = {
    'n_estimators': int(best_ns[9]),
    'booster': 'gbtree',
    'tree_method': 'hist',
    'random_state': 0,
    'n_jobs': -1,
    'max_depth': int(optimizer.max['params']['max_depth']),
    'subsample': optimizer.max['params']['subsample'],
    'colsample_bytree': optimizer.max['params']['colsample_bytree'],
    'reg_alpha': optimizer.max['params']['reg_alpha'],
    'reg_lamda': optimizer.max['params']['reg_lamda'],
    'metric': 'error'
}

reg = XGBRegressor(**best_params, objective='reg:squarederror').fit(train_X, train_y)


In [14]:
train_X

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Functional_Maj2
0,0.073375,-0.274183,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.605161,0.583521,-0.236417,...,0,0,1,0,0,0,0,1,0,1.0
1,-0.872563,0.466980,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.594237,1.182569,-0.274287,...,0,0,1,0,0,0,0,1,0,1.0
2,0.073375,-0.119268,0.073480,0.651479,-0.517200,0.984752,0.830215,0.293825,0.073564,-0.248776,...,0,0,1,0,0,0,0,1,0,1.0
3,0.309859,-0.496292,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.538429,-0.528276,-0.321845,...,0,0,1,1,0,0,0,0,0,1.0
4,0.073375,0.754272,0.375148,1.374795,-0.517200,0.951632,0.733308,1.353308,0.450261,-0.236424,...,0,0,1,0,0,0,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.073375,-0.216180,-0.260560,-0.071836,-0.517200,0.918511,0.733308,-0.695338,-0.969538,-0.271354,...,0,0,1,0,0,0,0,1,0,1.0
1456,-0.872563,0.579019,0.266407,-0.071836,0.381743,0.222975,0.151865,0.036785,0.807385,0.599934,...,0,0,1,0,0,0,0,1,0,1.0
1457,0.309859,-0.061878,-0.147810,0.651479,3.078570,-1.002492,1.024029,-0.836983,-0.153404,-0.199858,...,0,0,1,0,0,0,0,1,0,1.0
1458,-0.872563,-0.215476,-0.080160,-0.795151,0.381743,-0.704406,0.539493,-0.524256,-0.891485,6.033440,...,0,0,1,0,0,0,0,1,0,1.0


In [15]:
ans = reg.predict(test_X.values)

ValueError: feature_names mismatch: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'LotShape_IR1', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_AllPub', 'Utilities_NoSeWa', 'LotConfig_Corner', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Gtl', 'LandSlope_Mod', 'LandSlope_Sev', 'Neighborhood_Blmngtn', 'Ne

In [ ]:
ans

In [ ]:
ans = (ans * np.std(train_y)[0]) + np.mean(train_y)[0]

In [ ]:
ans

In [ ]:
sub.to_csv('./sub/sub6.csv', index=False)

In [16]:
train_X.columns == test_X.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,